<a href="https://colab.research.google.com/github/dhanu0809/Shikshalens/blob/main/working_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We start by reading the Excel file with pandas

In [ ]:
import pandas as pd
import numpy as np

# Read the Excel data (adjust the filename/path as needed)
df = pd.read_excel("student_dt_classwise_indian_teachers (1).xlsx")

# Take a look at the first few rows and summary information
print(df.head())
print(df.info())


  Student ID        Student Name Student ID.1  Attendance (%)  Internal Marks  \
0       S001       Himmat Sehgal         S001           56.22              16   
1       S002      Reyansh Tailor         S002           96.55              26   
2       S003           Tanya Ben         S003           81.24              29   
3       S004        Yasmin Reddy         S004           71.91              38   
4       S005  Dharmajan Dhaliwal         S005           40.92              13   

   External Marks  Total Marks Grade  Gender  Age  ... Toilet Facilities  \
0              51           67     B    Male   12  ...               Yes   
1              25           51     C  Female   11  ...               Yes   
2              51           80     B    Male   12  ...               Yes   
3              23           61     B  Female   11  ...               Yes   
4              30           43     C  Female   10  ...               Yes   

  Library Guardian Contact Preferred Mode           Fath

Since the dataset includes columns such as student names, IDs, and other non-predictive features, remove columns that are not useful for prediction. Also, check for missing values, clean data, and fix any inconsistencies.


In [ ]:
df.describe()

,Attendance (%),Internal Marks,External Marks,Total Marks,Age,Digital Learning Hours,Homework Submission Rate,Disciplinary Records,Student-Teacher Ratio,Guardian Contact
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000,5.000000e+02
mean,72.629400,24.912000,40.770000,65.682000,11.906000,6.83400,75.523780,1.638000,40.074000,9.490842e+09
std,17.082218,9.072203,12.046382,15.117074,1.370007,4.36997,18.493879,1.436343,11.203909,2.903547e+08
min,30.390000,10.000000,20.000000,30.000000,10.000000,0.00000,28.140000,0.000000,20.000000,9.006724e+09
25%,62.457500,16.750000,30.000000,55.000000,11.000000,3.00000,63.372500,0.000000,31.000000,9.246982e+09
50%,73.600000,26.000000,41.000000,66.000000,12.000000,7.00000,76.775000,1.000000,39.000000,9.486857e+09
75%,85.397500,33.000000,51.000000,76.000000,13.000000,11.00000,90.207500,3.000000,50.000000,9.745784e+09
max,100.000000,39.000000,69.000000,108.000000,14.000000,14.00000,100.000000,4.000000,59.000000,9.991747e+09


In [ ]:
# Check missing values
print(df.isnull().sum())

# For demonstration, assume missing values in numeric columns can be imputed with the mean
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# For categorical columns, we can fill missing entries with "Unknown"
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].fillna("Unknown")


Student ID                    0
Student Name                  0
Student ID.1                  0
Attendance (%)                0
Internal Marks                0
External Marks                0
Total Marks                   0
Grade                         0
Gender                        0
Age                           0
Religion/Caste                0
BPL Status                    0
Parent Education Level      183
Family Income Range           0
DIKSHA Access                 0
Digital Learning Hours        0
Participation in Quiz         0
Homework Submission Rate      0
Disciplinary Records          0
School Name                   0
District                      0
Student-Teacher Ratio         0
Electricity                   0
Toilet Facilities             0
Library                       0
Guardian Contact              0
Preferred Mode                0
Father Name                   0
Mother Name                   0
Class                         0
Section                       0
Class-Se

If you don’t already have a column indicating dropout risk, you need to create one. For demonstration, we’ll define dropout risk as a binary value where students with attendance below (for example) 60% are marked as “at risk.” You might base this on multiple factors (e.g., low marks and high disciplinary records) in a real scenario.

In [ ]:
# Create a synthetic dropout risk variable based on Attendance (%)
# You might also want to combine other variables (like Total Marks, Disciplinary Records, etc.)
df['Dropout_Risk'] = np.where(df['Attendance (%)'] < 60, 1, 0)

Categorical features must be encoded. Numerical features may be scaled. One simple approach is to use label encoding or one-hot encoding for the categorical variables.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Identify features (X) and target (y)
y = df['Dropout_Risk']
X = df.drop(columns=['Dropout_Risk'])

# For simplicity, let’s use one-hot encoding for categorical variables.
X_encoded = pd.get_dummies(X, drop_first=True)

# Normalize numerical features
scaler = StandardScaler()
X_encoded.loc[:, X_encoded.select_dtypes(include=['float64', 'int64']).columns] = scaler.fit_transform(
    X_encoded.select_dtypes(include=['float64', 'int64']))

<ipython-input-11-973820273b74>:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.98332516  0.12004688  0.45105849  1.44409332 -1.31433677  1.55443053
 -1.20399957  0.7820701  -0.98332516 -0.32130194 -0.10062753 -0.54197635
  1.44409332 -1.31433677 -0.32130194 -0.98332516  1.22341892  0.34072129
  0.6717329   1.33375612 -1.53501118 -0.65231355 -0.32130194  1.55443053
  1.00274451  0.56139569 -1.09366236  1.33375612  1.33375612 -0.43163914
 -0.43163914  0.45105849  1.55443053  1.55443053 -0.54197635  1.11308171
  0.7820701   1.33375612  1.00274451 -0.98332516  1.55443053 -1.64534838
 -1.64534838  1.00274451  1.22341892  1.55443053  1.00274451  0.45105849
 -0.32130194 -0.76265075 -1.42467398 -0.98332516 -1.09366236 -0.87298796
  1.22341892 -0.76265075  1.55443053 -1.20399957 -1.64534838  0.34072129
 -0.65231355 -0.43163914  0.8924073  -0.10062753  1.22341892  0.6717329
  0.8924073  -0.76265075  0.45105849  0.1200

We will split the data into training and testing sets and then use, for example, a Random Forest Classifier for prediction

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

Once you have risk scores for each student, you could design an intervention service. For instance:

A simple (rule-based) recommendation engine might look like this:

In [ ]:
def recommend_intervention(row):
    if row['Attendance (%)'] < 60 and row['Total Marks'] < 50:
        return "Provide academic tutoring and counseling."
    elif row.get('Disciplinary Records', 0) > 0:
        return "Signpost to behavioral intervention programs."
    elif row.get('Digital Learning Hours', 0) < 5:
        return "Offer extra digital learning support."
    else:
        return "Monitor and encourage engagement."

# For the test set (or the entire dataset) add recommendations:
df['Intervention'] = df.apply(recommend_intervention, axis=1)
print(df[['Attendance (%)', 'Total Marks', 'Digital Learning Hours', 'Disciplinary Records', 'Intervention']].head())

   Attendance (%)  Total Marks  Digital Learning Hours  Disciplinary Records  \
0           56.22           67                       8                     0   
1           96.55           51                       1                     0   
2           81.24           80                       7                     1   
3           71.91           61                       7                     1   
4           40.92           43                       6                     0   

                                    Intervention  
0              Monitor and encourage engagement.  
1          Offer extra digital learning support.  
2  Signpost to behavioral intervention programs.  
3  Signpost to behavioral intervention programs.  
4      Provide academic tutoring and counseling.  


For a dataset with diverse academic, demographic, and behavioral features as you have, starting with a tree-based ensemble method such as Random Forest or XGBoost is a solid choice. These algorithms are robust in handling missing values, unscaled data, and feature interactions, and they provide feature importance metrics which are valuable in an educational context for interpreting which factors most influence dropout risk.


In [ ]:
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# --- Classifiers ---
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

# ---------------------------
# Define the Classifiers to Compare
# ---------------------------
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Gaussian NB": GaussianNB(),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# ---------------------------
# 6. Compare Models Using Cross-Validation
# ---------------------------
# We will use 5-fold Stratified CV and compute accuracy, ROC-AUC, and F1-score.
scoring = {
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc',
    'f1': 'f1'
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to hold results
results = {}

for name, model in models.items():
    cv_results = cross_validate(model, X_encoded, y, cv=cv, scoring=scoring, n_jobs=-1)
    results[name] = {
        'accuracy': np.mean(cv_results['test_accuracy']),
        'roc_auc': np.mean(cv_results['test_roc_auc']),
        'f1': np.mean(cv_results['test_f1'])
    }
    print(f"{name} results:")
    print(f"  Accuracy: {results[name]['accuracy']:.3f}")
    print(f"  ROC AUC : {results[name]['roc_auc']:.3f}")
    print(f"  F1 Score: {results[name]['f1']:.3f}\n")

# Optionally, display the comparison in a DataFrame:
results_df = pd.DataFrame(results).T
print("Comparison of Model Performance:")
print(results_df)

Logistic Regression results:
  Accuracy: 0.962
  ROC AUC : 0.996
  F1 Score: 0.902

Random Forest results:
  Accuracy: 0.926
  ROC AUC : 1.000
  F1 Score: 0.774

SVM results:
  Accuracy: 0.944
  ROC AUC : 0.992
  F1 Score: 0.836

Gradient Boosting results:
  Accuracy: 1.000
  ROC AUC : 1.000
  F1 Score: 1.000

Gaussian NB results:
  Accuracy: 0.264
  ROC AUC : 0.525
  F1 Score: 0.355

XGBoost results:
  Accuracy: 0.996
  ROC AUC : 0.999
  F1 Score: 0.990

Comparison of Model Performance:
                     accuracy   roc_auc        f1
Logistic Regression     0.962  0.995683  0.901754
Random Forest           0.926  0.999629  0.774126
SVM                     0.944  0.992486  0.836422
Gradient Boosting       1.000  1.000000  1.000000
Gaussian NB             0.264  0.524876  0.354526
XGBoost                 0.996  0.998614  0.990471


Gradient Boosting

In [ ]:
# Train the Gradient Boosting Model
# ---------------------------
# Define the classifier with hyperparameters:
# n_estimators: number of weak learners (trees)
# learning_rate: step size (shrinkage) for each tree's contribution
# max_depth: maximum depth of each individual tree
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                 max_depth=3, random_state=42)

# Train the model
gbc.fit(X_train, y_train)

# ---------------------------
# 7. Model Evaluation
# ---------------------------
# Predict on the test set
y_pred = gbc.predict(X_test)
y_pred_proba = gbc.predict_proba(X_test)[:, 1]

# Print out the classification report and ROC AUC Score
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC AUC Score: {:.3f}".format(roc_auc_score(y_test, y_pred_proba)))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        73
           1       1.00      1.00      1.00        27

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

ROC AUC Score: 1.000


# XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# ---------------------------
# Train the XGBoost Model
# ---------------------------
# We disable the label encoder and set eval_metric to logloss
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# ---------------------------
# Model Evaluation
# ---------------------------
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))
print("ROC AUC Score: {:.3f}".format(roc_auc_score(y_test, y_pred_proba)))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:38:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        73
           1       0.90      1.00      0.95        27

    accuracy                           0.97       100
   macro avg       0.95      0.98      0.96       100
weighted avg       0.97      0.97      0.97       100

ROC AUC Score: 0.979


In [ ]:
# ---------------------------
# Combine User Details with Predictions
# ---------------------------
# Since the train/test split retains the original indices, we can use these indices to retrieve user details.
# Let's assume that user_details was created from the original data.
results_df = df.loc[X_test.index].copy()
results_df["Actual_Risk"] = y_test
results_df["Predicted_Risk"] = y_pred

# Optionally, if you wish to see probability information as well, add:
results_df["Predicted_Prob"] = y_pred_proba

print("\nOutput with User Details and Predictions:")

cols_to_drop = ['Internal Marks', 'External Marks', 'Total Marks','Class','Parent Education Level','Student ID.1',
                'Religion/Caste',  'District','Toilet Facilities','Library','Student-Teacher Ratio',' Student-Teacher Ratio','Section','Electricity','Family Income Range','School Name']
results_df = results_df.drop(columns=cols_to_drop, errors='ignore')
print(results_df)
# Optionally, save the results to an Excel or CSV file.
# results_df.to_excel("dropout_predictions.xlsx", index=False)
# results_df.to_csv("dropout_predictions.csv", index=False)


Output with User Details and Predictions:
    Student ID        Student Name  Attendance (%) Grade  Gender  Age  \
361       S362       Leslie Peters           52.01     B  Female   14   
73        S074         Vivaan Shah           87.08     A  Female   11   
374       S375     Amber Velazquez          100.00     B  Female   13   
155       S156        Colleen Paul           57.83     B  Female   13   
104       S105        James Becker           81.26     B    Male   11   
..         ...                 ...             ...   ...     ...  ...   
347       S348        Jeffrey Best           75.90     C    Male   11   
86        S087   Inaaya  Chowdhury           81.07     B    Male   10   
75        S076         Seher Gupta           81.03     B    Male   12   
438       S439  Mr. Tanner Beltran           53.78     B  Female   12   
15        S016            Azad Som           42.84     B    Male   13   

    BPL Status DIKSHA Access  Digital Learning Hours Participation in Quiz  \
36

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# ---------------------------
# 1. Load Your Data (Assuming you already have 'df' loaded)
# ---------------------------

print("Original Data (sample):")
print(df.head())

# ---------------------------
# 2. Data Cleaning and Preprocessing
# ---------------------------
# (a) Drop non-predictive or personal information columns.
# (b) Fill missing values:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=[object]).columns.tolist()
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())
df[cat_cols] = df[cat_cols].fillna("Unknown")

# (c) (Optional) If your training also created a target column, ignore or drop it:
if 'Dropout_Risk' in df.columns:
    data_clean = df.drop(columns=['Dropout_Risk'])
    y = df['Dropout_Risk'] # Extract the target variable
else:
    # If 'Dropout_Risk' is not in df, you'll need to define 'y' based on your logic
    # For example, if you're using attendance as a proxy:
    y = np.where(df['Attendance (%)'] < 60, 1, 0)

# ---------------------------
# 3. Feature Engineering: One-Hot Encoding & Scaling
# ---------------------------
# One-Hot Encode categorical variables.
X_encoded = pd.get_dummies(df, drop_first=True)

# Initialize the StandardScaler and OneHotEncoder
scaler = StandardScaler()
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for compatibility with older versions of scikit-learn

# Identify numeric and categorical features
numeric_features = X_encoded.select_dtypes(include=[np.number]).columns
categorical_features = X_encoded.select_dtypes(include=[object]).columns

# Fit and transform numeric features
X_encoded[numeric_features] = scaler.fit_transform(X_encoded[numeric_features])

# Save the scaler for later use
joblib.dump(scaler, 'scaler.pkl')

# ---------------------------
# 4. (Re-)Train and Save the Model
# ---------------------------
# Assuming you want to use XGBoost
from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_encoded, y) # Fit the model to your entire preprocessed dataset

# Save the trained model to a file using joblib.dump
joblib.dump(model, "dropout_model.pkl")

# ---------------------------
# 5. Load the Trained Model
# ---------------------------
# (Now this should work without errors)
model = joblib.load("dropout_model.pkl")

# ---------------------------
# 6. Generate Predictions for the Entire Dataset
# ---------------------------
df["Predicted_Risk"] = model.predict(X_encoded)
df["Predicted_Probability"] = model.predict_proba(X_encoded)[:, 1]

# ---------------------------
# 7. Output the Results
# ---------------------------
print("\nData with Predictions:")
print(df.head())

df.to_excel("student_predictions.xlsx", index=False)

Original Data (sample):
  Student ID        Student Name Student ID.1  Attendance (%)  Internal Marks  \
0       S001       Himmat Sehgal         S001           56.22              16   
1       S002      Reyansh Tailor         S002           96.55              26   
2       S003           Tanya Ben         S003           81.24              29   
3       S004        Yasmin Reddy         S004           71.91              38   
4       S005  Dharmajan Dhaliwal         S005           40.92              13   

   External Marks  Total Marks Grade  Gender  Age  ... Toilet Facilities  \
0              51           67     B    Male   12  ...               Yes   
1              25           51     C  Female   11  ...               Yes   
2              51           80     B    Male   12  ...               Yes   
3              23           61     B  Female   11  ...               Yes   
4              30           43     C  Female   10  ...               Yes   

  Library Guardian Contact Prefe

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:28:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Data with Predictions:
  Student ID        Student Name Student ID.1  Attendance (%)  Internal Marks  \
0       S001       Himmat Sehgal         S001           56.22              16   
1       S002      Reyansh Tailor         S002           96.55              26   
2       S003           Tanya Ben         S003           81.24              29   
3       S004        Yasmin Reddy         S004           71.91              38   
4       S005  Dharmajan Dhaliwal         S005           40.92              13   

   External Marks  Total Marks Grade  Gender  Age  ... Guardian Contact  \
0              51           67     B    Male   12  ...       9014475712   
1              25           51     C  Female   11  ...       9592551615   
2              51           80     B    Male   12  ...       9021637593   
3              23           61     B  Female   11  ...       9486986073   
4              30           43     C  Female   10  ...       9262754377   

  Preferred Mode           Father Name